In [1]:
#Name: Priyanshi
#Roll Number: 18IM30031

In [2]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## Given Data

In [3]:
transac_rate_rT=0.01                             #Transaction Cost Rate
depo_intrst_rate_rD=0.009                        #Deposit Interest Rate
Deposit=60                                       #Deposit
K=0.15                                           #Reserve Ratio
L=[10,25,4,11,18,3,17,15,9,10]                   #Loan Size
rL=[0.021,0.022,0.021,0.027,0.025,0.026,0.023,0.021,0.028,0.022]    #Loan Interest Rate
loss=[0.0002,0.0058,0.0001,0.0003,0.0024,0.0002,0.0058,0.0002,0.001,0.001]   
LEN=len(loss)


# Parameters

In [4]:
#SA Parameters
Cooling_Factor=0.9
Initial_Temperature=800
Current_Temp_Iter= 12
Rejection_Limit=5     #Rejection Counter Limit in Simulated Annealing

#GA Parameters
Population_Size=12    
GA_max_Iter=15
Crosover_Prob=0.8     #CrossOver Probability
Mutation_Prob=0.006   #Mutation Probability

In [5]:
#Initialising Population

def Population_Initialisation(Population_Size,n,L,K,Deposit):
    Population=[]
    for i in range(Population_Size):
        Chromo=[]
        j=0
        while j< LEN:
            
            Chromo.append(random.choice([0,1]))
            j+=1
        Population.append(Chromo)
    
    return Population

In [6]:
#Calculating Transaction Cost

def Insti_TC(K,Deposit,L):
    
    TC= (1-K)*Deposit-L
    return TC

In [7]:
#Calculating Loan Revenue Generated
#LR=Loan_Revenue
def LR_nu(LEN,rL,L,loss,Chromo):
    LR=0
    for i in range(LEN):
        LR+=(rL[i]*L[i]-loss[i])*Chromo[i]
    return LR

#Calculating cost of Demand Deposit
#DD=Demand Deposit
def DD_beta( depo_intrst_rate_rD,Deposit):
    return  depo_intrst_rate_rD*Deposit

In [8]:
#Calculating sum of all individual Loans

def Loan_Total_Sum(LEN,Chromo,L):
    LoanTotal=0
    for i in range(LEN):
        LoanTotal+=Chromo[i]*L[i]
    
    return LoanTotal

In [9]:
#Calculating Total Transaction Cost of expected lending Decision
#TTC=Total Transaction Cost
def TTC_omega(LEN,transac_rate_rT,K,Deposit,L,Chromo):
    TTC=0
    
    for i in range(LEN):
        TTC+=  transac_rate_rT*Insti_TC(K,Deposit,L[i])*Chromo[i]
    
    return TTC

#Checking the Constraint [total_loan>(1-K)*D] If satisfied or not and generating new Chromosome if satisfied
#If constraint violated randomly choosing chromosome bits

def Constraint_Evaluation(LEN,Chromo,L,K,Deposit):
    while(1):
        if (1-K)*Deposit <Loan_Total_Sum(LEN,Chromo,L):
            Chromo=[]
            
            for j in range(LEN):
                Chromo.append(random.choice([0,1]))
        else:
            break
   
    return Chromo

In [10]:
#Calling Constraint Check  and modifying Population if constrint is niot satisfied

def ImproveChromo(LEN,Population_Size,Population,L,K,Deposit):
    i=0
    while i<Population_Size:
        Population[i] = Constraint_Evaluation(LEN,Population[i],L,K,Deposit)
        i+=1
    return Population

In [11]:
def FitVal_Func(LEN,rL,  transac_rate_rT, depo_intrst_rate_rD,Deposit,K,L,loss,Population):
    FitVals=[]
    
    for i in range(Population_Size):
        FitVals.append(Fitness_Function(LEN,rL,  transac_rate_rT, depo_intrst_rate_rD,Deposit,K,L,  loss,Population[i]))
    
    return FitVals

#Fitness Function [Fx=nu+omega-beta-total_loss_lambda]

def Fitness_Function(LEN,rL,  transac_rate_rT, depo_intrst_rate_rD,Deposit,K,L,loss,Chromo):
    loss_lambda=0
    for i in range(LEN):
          loss_lambda+= loss[i]*Chromo[i]
    return LR_nu(LEN,rL,L,loss,Chromo)+ TTC_omega(LEN,transac_rate_rT,K,Deposit,L,Chromo)-DD_beta(depo_intrst_rate_rD,Deposit)-  loss_lambda

In [12]:
def GA_Roulette(Population_Size,FitVals,Population):
    prob=[]
    P=[]
    SUM_Fitness=sum(FitVals)
    ctr=0
    for i in range(Population_Size):
        ctr+=FitVals[i]
        prob.append(ctr/SUM_Fitness)

    for i in range(Population_Size):
        rand_num=random.random()
        for j in range(Population_Size):
            if rand_num < prob[j]:
                P.append(Population[j])
                break
    return P

In [13]:

def GA_Mutation(LEN,Population_Size,Mutation_Prob,Parents):
    i=0
    while i<Population_Size:
        for j in range(LEN):
            rnd=random.random()
            
            if rnd < Mutation_Prob:
                if Parents[i][j]==1:
                    Parents[i][j]=0
                else:
                    Parents[i][j]=1
        i+=1
    
    return Parents

def GA_Crossover(Population_Size,LEN,Parents,Crosover_Prob):
    i=0
    if Population_Size%2==0:  #if population Size is even
        
        while(i<Population_Size):
            Parents[i],Parents[i+1]=One_Point_Cross(LEN,Parents[i],Parents[i+1],Crosover_Prob)
            i+=2
        
        return Parents
   
    #if population size is odd
    else:
       
        while(i<Population_Size-1):
            Parents[i],Parents[i+1]=One_Point_Cross(LEN,Parents[i],Parents[i+1],Crosover_Prob)
            i+=2
        return Parents

In [14]:
def One_Point_Cross(LEN,P1,P2,Crosover_Prob):
    rand_num=random.random()
    if rand_num < Crosover_Prob:
        C1=[]  #child1
        C2=[]  #chid2
        Crossover_Point=random.choice([i for i in range(1,LEN)])
        #Changing the chromosome bits after crossover point
        for j in range(LEN):
            if j < Crossover_Point:
                C1.append(P1[j])
                
                C2.append(P2[j])
            
            else:
                C1.append(P2[j])
                
                C2.append(P1[j])
        
        return C1,C2
    
    else:
        return P1,P2

In [15]:
def Genetic_Algorithm(LEN,Population_Size,GA_max_Iter,rL,transac_rate_rT, depo_intrst_rate_rD,Deposit,K,L,  loss,Population):
    Population=ImproveChromo(LEN,Population_Size,Population,L,K,Deposit)
    
    #Initialising globest value as 0 and GlobalBest Solution as [].
    GBestSol=[]
    GBest=0
    
    for i in range(GA_max_Iter):
        #Calculating Fitness Values
        
        FitVals = FitVal_Func(LEN,rL,  transac_rate_rT, depo_intrst_rate_rD,Deposit,K,L, loss,Population) 
        
        GBest,GBestSol = UpdateGBest(Population,FitVals,GBest,GBestSol)
        
        #Roulette Wheel Selection and do crossover and mutation
       
        Population=GA_Roulette(Population_Size,FitVals,Population)
        Population=GA_Crossover(Population_Size,LEN,Population,Crosover_Prob)
        Population=GA_Mutation(LEN,Population_Size,Mutation_Prob,Population)
        
        #Changing ChromoSomes according to constraint
        Population=ImproveChromo(LEN,Population_Size,Population,L,K,Deposit)
    
    FitVals=FitVal_Func(LEN,rL,  transac_rate_rT, depo_intrst_rate_rD,Deposit,K,L,  loss,Population)
    GBest,GBestSol = UpdateGBest(Population,FitVals,GBest,GBestSol)
    
    return Population,GBest,GBestSol

def UpdateGBest(Population,FitVals,GBest,GBestSol):
    Currval=max(FitVals)

    #If current solution is better than global solution changing current solution to global solution 
    
    if Currval > GBest:
        GBest=Currval
        
        GBestSol=Population[FitVals.index(max(FitVals))]
    
    return GBest,GBestSol

In [16]:
e=2.71

# To check Results of GA only.

In [17]:
Population=Population_Initialisation(Population_Size,LEN,L,K,Deposit)

In [18]:
Pop,val,sol=Genetic_Algorithm(LEN,Population_Size,GA_max_Iter,rL,transac_rate_rT, depo_intrst_rate_rD,Deposit,K,L,  loss,Population)
print('Bank Profit',val)
print('Lending Decision using GA',sol)

Bank Profit 2.5934
Lending Decision using GA [0, 0, 1, 0, 0, 1, 1, 1, 0, 1]


# Amalgamtion of Simulated Annealing and GA

In [19]:
def Simulated_Annealing(Initial_Temperature,Cooling_Factor,Current_Temp_Iter,Rejection_Limit,LEN,Population_Size,GA_max_Iter,rL,  transac_rate_rT, depo_intrst_rate_rD,Deposit,K,L,  loss):
    
    #Setting temperature to initial temperature
    Temp=Initial_Temperature
    
    #Initialising population
    Population=Population_Initialisation(Population_Size,LEN,L,K,Deposit)
    
    GBest=0
    GBestSol=[]
    #Keeping lowest Temperature as room temperature
    while Temp > 25:
        Rejection_Ctr=0   #Rejection counter to count number of times the solution rejected
        i=0
        while i<Current_Temp_Iter:
            
            New_Pop,CurrentValue,CurrentSol=Genetic_Algorithm(LEN,Population_Size,GA_max_Iter,rL,  transac_rate_rT, depo_intrst_rate_rD,Deposit,K,L,loss,Population)
            rand_num=random.random()

            #Updating GlobalBest Function Value
            if CurrentValue>GBest:
                
                GBest=CurrentValue
                GBestSol=CurrentSol
                Population= New_Pop
            
            #Accepting worse solution
            elif rand_num < np.power(e,((GBest-CurrentValue)/Temp)):
                Population= New_Pop
            
            else:
                Rejection_Ctr+=1

            if Rejection_Ctr==Rejection_Limit:
                break
            
            i+=1
        Temp=Cooling_Factor*Temp   #Updating Temperature
    
    
    print('Global Best Function Value or Bank Profit',GBest)
    print('Global Best Solution or Lending Decision',GBestSol)

In [20]:
Simulated_Annealing(Initial_Temperature,Cooling_Factor,Current_Temp_Iter,Rejection_Limit,LEN,Population_Size,GA_max_Iter,rL,transac_rate_rT, depo_intrst_rate_rD,Deposit,K,L,loss)

Global Best Function Value or Bank Profit 3.1854
Global Best Solution or Lending Decision [1, 0, 1, 1, 0, 1, 0, 0, 1, 1]
